In [ ]:
%pylab inline
rcParams['figure.figsize'] = (15, 4)

In [ ]:
from scipy.signal import freqz

In [ ]:
phs = linspace(0, 20 * 2 * pi, 1024, endpoint=False)
g = cos(phs + 0.4)
plot(g)
pass

In [ ]:
plot(abs(fft.rfft(g[:512], n=512)))
xlim(0, 40)
pass

In [ ]:
plot(abs(fft.rfft(g[512:1024], n=512)))
xlim(0, 40)
pass

In [ ]:
X = abs(fft.rfft(g[512:1024], n=512))
argmax(X)

In [ ]:
fft.fftfreq(512, 1/1024)[10] # Hz

Starting phase for first window:

In [ ]:
angle(fft.rfft(g[:512])[10])

Starting phase for second window: (no phase change!)

In [ ]:
angle(fft.rfft(g[512:1024])[10])

Phase at the end of the window:

In [ ]:
0.4 + (20 * 2 * pi)

If we wrap (think modulo) the phase:

In [ ]:
(0.4 + (20 * 2 * pi)) % (2 * pi)

However for a frequency that has a fractional cycle the ending window phase is:

In [ ]:
(20.2 * 2 * pi)

In [ ]:
(20.2 * 2 * pi) - (0.4 + (20 * 2 * pi))

And wrapped:

In [ ]:
(20.2 * 2 * pi) % (2 * pi)

Let's see a concrete example:

In [ ]:
# 20 Hz ----> 23.5 Hz
phs = linspace(0, 23.5 * 2 * pi, 1024, endpoint=False)
g = cos(phs)
plot(g)
pass

In [ ]:
stem(abs(fft.rfft(g[:512], n=512)))
xlim(0, 40)
pass

In [ ]:
X = abs(fft.rfft(g[:512], n=512))
argmax(X)

In [ ]:
fft.fftfreq(512, 1/1024)[12] # Hz

But we know that the precise center is at 23.5 Hz! Are we loosing information? Where's the information about the precise frequency?

Because the number of cycles is not an integer, the starting phase for the second window is different:

In [ ]:
plot(g[:512])
plot(g[512:1024])
pass

Starting phase first window:

In [ ]:
angle(fft.rfft(g[:512]))[12]

Starting phase second window:

In [ ]:
angle(fft.rfft(g[512:1024]))[12]

Phase difference:

In [ ]:
difference = angle(fft.rfft(g[512:1024]))[12] - angle(fft.rfft(g[:512]))[12]
difference

The total elapsed phase if our signal were perfectly aligned on the bin:

In [ ]:
24 * 2 * pi

But what actually elapsed (adding the phase difference measured) was:

In [ ]:
24 * 2 * pi + difference * 2 # there needs to be a factor of 2 in there

Which if we count cycles in that phase accumulation we get:

In [ ]:
(24 * 2 * pi + difference * 2) / (2 * pi)

Our missing frequency precision is hidden in the phase!!! If we keep track of phase then we'll have better info on frequency. Remember that: phase is the integral of frequency. frequency is the derivative of phase. phase is the accumulation of frequency. frequency is the rate of change of phase.

# Phase Vocoder

Assumptions: there is 1 sin wave per bin with continuously accumulating phase. This is not always a valid assumption.

Two sine tones, the second with 4 times the frequency and half the amplitude..

In [ ]:
f0 = 203.3
p0 = 0.3
N = 4096

test_phs = linspace(0, (f0 * 2 * pi), N, endpoint=False)
test_sig = sin(test_phs + p0) + 0.5 * cos(p0 + (test_phs * 4))

In [ ]:
#X = rfft(test_sig[:512] * hanning(512)) # first window
X = rfft(test_sig[:512])
plot(abs(X))
pass

In [ ]:
#X = rfft(test_sig[:512] * hanning(512))
X = rfft(test_sig[:512])
stem(abs(X))
xlim(10, 40)
pass

In [ ]:
argmax(abs(X)), fft.fftfreq(512, 1/4096)[25] # Hz

In [ ]:
(argmax(abs(X)) + 1), fft.fftfreq(512, 1/4096)[26] # Hz

The frequency we are looking for is in between. Let's do STFT with overlap:

In [ ]:
# 4 * H = W
H = 128
W = 512
window = hanning(W)
stft = []

for i in linspace(0, len(test_sig) - W, H):
    i = int(i)
    X = rfft(test_sig[i:i + W] * window)
    stft.append(X)

In [ ]:
argmax(abs(stft[1]))

In [ ]:
stem(abs(stft[1]))
xlim(10, 40)
figure()
stem(abs(stft[5]))
xlim(10, 40)
figure()
stem(abs(stft[9]))
xlim(10, 40)

We're not seeing any change in the magnitude spectrum. Let's look at the phase...

In [ ]:
imshow(angle(stft).T, aspect='auto', cmap=cm.gray, interpolation='nearest')
colorbar()
pass

In [ ]:
stem(angle(stft[1]))
xlim(10, 40)
figure()
stem(angle(stft[5]))
xlim(10, 40)
figure()
stem(angle(stft[9]))
xlim(10, 40)

In [ ]:
stem(angle(stft).T[24])
pass

In [ ]:
stem(angle(stft).T[25])
pass

In [ ]:
stem(angle(stft).T[26])
pass

Let's look at the unwrapped phase:

In [ ]:
# start with the phase in bin 25 from the initial (0th) STFT
unwrapped = [angle(stft).T[25][0]]

for p in angle(stft).T[25][1:]:
    while (p < unwrapped[-1]):
        p += 2 * pi
    unwrapped.append(p)
plot(unwrapped, '.')
pass

A line! The frequency is constant across all the bins. What the slope of this line?

In [ ]:
4 * ((unwrapped[-1] - unwrapped[0]) / (2 * pi)), len(test_sig)

slope = rise / run. The slope of the line is related to the frequency

In [ ]:
delta_phase = diff(unwrapped)

In [ ]:
stem(delta_phase)
pass

Now accumulate phase for the hop period:

In [ ]:
fft.fftfreq(512, 1/4096)[25] # Hz --- > cycles per second

In [ ]:
k = 25 # bin number
cycles = k * (H / W) # how many cycles of bin between STFT hops
phase_per_hop = cycles * 2 * pi # Total accumulated phase for bin
cycles, phase_per_hop

For a perfectly bin-aligned sin wave, we would expect the above to be true.

Difference in phase of bin 25 between the first 2 STFTs:

In [ ]:
delta_phase_0 = delta_phase[0]
delta_phase_0

Expected from a perfectly bin-aligned sin wave:

In [ ]:
phase_per_hop % (2 * pi)

Phase divergence:

In [ ]:
divergence = delta_phase_0 - phase_per_hop % (2 * pi)
divergence

In [ ]:
corrected_phase = phase_per_hop + divergence
corrected_phase

Estimate number of cycles in hop period from total accumulated phase:

In [ ]:
corrected_phase / (2 * pi)

In [ ]:
cycles # compare to this

Multiply by overlap:

In [ ]:
(W / H) * corrected_phase / (2 * pi)

Now times the number of windows to match the original full length:

In [ ]:
(N / W) * (W / H) * corrected_phase / (2 * pi)

Yay! (or not...!) 

Original frequency was 203.3, but this still got us much closer to the original frequency than what the FFT bins provide:

In [ ]:
(N / W) * 25

In [ ]:
(N / W) * 26

By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

Adapted by Karl Yerkes

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)